In [55]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e11/sample_submission.csv
/kaggle/input/playground-series-s4e11/train.csv
/kaggle/input/playground-series-s4e11/test.csv


In [61]:
# Import Libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

# Load Dataset
train_df = pd.read_csv('/kaggle/input/playground-series-s4e11/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s4e11/test.csv')

# Drop the 'id' column from the training set
train_df = train_df.drop(columns=['id'])

# Train Data Preprocessing
numerical_cols = train_df.select_dtypes(include=['float64']).columns
imputer = SimpleImputer(strategy='median')
train_df[numerical_cols] = imputer.fit_transform(train_df[numerical_cols])

categorical_cols = train_df.select_dtypes(include=['object']).columns
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col].astype(str))
    label_encoders[col] = le

# Prepare training features and target variable
X_train = train_df.drop(columns=['Depression'])
y_train = train_df['Depression']

# Test Data Preprocessing
X_test = test_df.drop(columns=['id'])
X_test[numerical_cols] = imputer.transform(X_test[numerical_cols])

for col in categorical_cols:
    le = label_encoders[col]
    X_test[col] = X_test[col].apply(lambda x: le.transform([x])[0] if x in le.classes_ else -1)

# Random Forest Model
rf_model = RandomForestClassifier(
    n_estimators=500, 
    max_depth=10, 
    min_samples_split=2, 
    min_samples_leaf=1, 
    random_state=42
)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

# Save RF Predictions
rf_submission = pd.DataFrame({
    'id': test_df['id'],
    'Depression': y_pred_rf
})
rf_submission.to_csv('/kaggle/working/rf_submission.csv', index=False)
print("Random Forest prediction file has been created")

# XGBoost Model
xgb_model = XGBClassifier(
    n_estimators=100, 
    learning_rate=0.1, 
    max_depth=6, 
    subsample=0.8, 
    colsample_bytree=0.8, 
    random_state=42
)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

# Save XGBoost Predictions
xgb_submission = pd.DataFrame({
    'id': test_df['id'],
    'Depression': y_pred_xgb
})
xgb_submission.to_csv('/kaggle/working/xgb_submission.csv', index=False)
print("XGBoost prediction file has been created")


Random Forest prediction file has been created
XGBoost prediction file has been created


pie chart of no.of depressed students and working professionals


In [62]:
'''Employee_depressed=Employee_data[Employee_data['Depression']==1].count()[0]
Student_depressed=student_data[student_data['Depression']==1].count()[0]
plt.pie([Employee_depressed,Student_depressed],labels=["depresseed employees","depressed students"],autopct='%.2f')
'''
df=pd.read_csv('/kaggle/working/xgb_submission.csv')


,id,Depression
0,140700,0
1,140701,0
2,140702,0
3,140703,1
4,140704,0
...,...,...
93795,234495,0
93796,234496,1
93797,234497,0
93798,234498,1
